In [1]:
# imposting libraries: 
import pandas as pd


In [2]:
# loading dataset: 

#llm_opinion/newdatasets/datasetgroup/output/tendims_final2.csv
#data= pd.read_csv("../llm_opinion/newdatasets/datasetgroup/output/tendims_final2.csv") 
data= pd.read_csv("./tendimension2_zeroshot.csv") 

data.columns.tolist()

FileNotFoundError: [Errno 2] No such file or directory: '../llm_opinion/newdatasets/datasetgroup/output/tendimension2_zeroshot.csv'

In [ ]:
# Basic information about the dataset
print("\nDataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nBasic statistics:")
print("\nTotal number of comments:", len(df))
print("Number of delta comments:", df['has_delta'].sum())
print("Delta rate: {:.2f}%".format((df['has_delta'].sum()/len(df))*100))


In [ ]:
data.dimension.values.count

In [ ]:
data.semantic_dimension.values.count

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_dimension_analysis(data):
    dimensions = ['support', 'knowledge', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'identity', 'romance']
    
    results = []
    for dim in dimensions:
        dim_col = f'{dim}_mean'
        delta_mean = data[data['has_delta'] == 1][dim_col].mean()
        non_delta_mean = data[data['has_delta'] == 0][dim_col].mean()
        
        # Calculate odds ratio
        median = data[dim_col].median()
        contingency = pd.crosstab(data[dim_col] > median, data['has_delta'])
        odds_ratio = (contingency.iloc[1,1] * contingency.iloc[0,0]) / \
                    (contingency.iloc[0,1] * contingency.iloc[1,0])
        
        results.append({
            'dimension': dim,
            'odds_ratio': odds_ratio,
            'delta_mean': delta_mean,
            'non_delta_mean': non_delta_mean
        })
    
    results_df = pd.DataFrame(results)
    
    # Set up the figure
    plt.figure(figsize=(12, 6))
    
    # Plot odds ratios
    sns.barplot(data=results_df, x='dimension', y='odds_ratio')
    plt.xticks(rotation=45)
    plt.title('Odds Ratios by Dimension')
    plt.tight_layout()
    plt.show()
    
    return results_df

# Run analysis and create plots
results = plot_dimension_analysis(data)
print("\nResults sorted by odds ratio:")
print(results.sort_values('odds_ratio', ascending=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def analyze_dimensions(data):
    dimensions = ['support', 'knowledge', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'identity', 'romance']
    
    results = []
    for dim in dimensions:
        dim_col = f'{dim}_mean'
        
        # Get means
        delta_mean = data[data['has_delta'] == 1][dim_col].mean()
        non_delta_mean = data[data['has_delta'] == 0][dim_col].mean()
        
        # Calculate odds ratio
        median = data[dim_col].median()
        high_score = data[dim_col] > median
        contingency = pd.crosstab(high_score, data['has_delta'])
        odds_ratio = (contingency.iloc[1,1] * contingency.iloc[0,0]) / \
                    (contingency.iloc[0,1] * contingency.iloc[1,0])
        
        results.append({
            'dimension': dim,
            'odds_ratio': odds_ratio,
            'delta_mean': delta_mean,
            'non_delta_mean': non_delta_mean
        })
    
    results_df = pd.DataFrame(results).sort_values('odds_ratio', ascending=False)
    
    # Create high-quality dot plot
    plt.figure(figsize=(12, 8), dpi=300)
    plt.scatter(results_df['odds_ratio'], 
               range(len(dimensions)),
               color=['blue' if x > 1 else 'red' for x in results_df['odds_ratio']], 
               s=100)
    
    # Formatting
    plt.yticks(range(len(dimensions)), results_df['dimension'], fontsize=12)
    plt.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    
    # Add values
    for i, row in results_df.iterrows():
        plt.text(row['odds_ratio'], i, f' {row["odds_ratio"]:.2f}', 
                va='center', fontsize=12)
    
    plt.xlabel('Odds Ratio', fontsize=12)
    plt.title('Social Dimensions Impact on Delta', fontsize=14, pad=20)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Print formatted table
    print("\nResults sorted by odds ratio:")
    pd.set_option('display.float_format', lambda x: '%.6f' % x)
    print(results_df[['dimension', 'odds_ratio', 'delta_mean', 'non_delta_mean']])
    
    return results_df, plt.gcf()

# Run analysis
results, fig = analyze_dimensions(data)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_odds_ratios(data):
    dimensions = ['knowledge', 'identity', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'support', 'romance']
    
    results = []
    for dim in dimensions:
        median = data[f'{dim}_mean'].median()
        contingency = pd.crosstab(data[f'{dim}_mean'] > median, data['has_delta'])
        
        try:
            odds_ratio = (contingency.iloc[1,1] * contingency.iloc[0,0]) / \
                        (contingency.iloc[0,1] * contingency.iloc[1,0])
        except:
            odds_ratio = np.nan
            
        results.append({
            'dimension': dim,
            'odds_ratio': odds_ratio
        })
    
    results_df = pd.DataFrame(results).sort_values('odds_ratio', ascending=False)
    
    # Visualization
    plt.figure(figsize=(12, 6), dpi=300)
    
    # Create horizontal bar plot
    bars = plt.barh(results_df['dimension'], results_df['odds_ratio'])
    plt.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    plt.xlabel('Odds Ratio')
    plt.title('Impact of Dimensions on Delta Likelihood')
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width, i, f' {width:.2f}', va='center')
    
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return results_df

results = analyze_odds_ratios(data)
print("\nResults sorted by odds ratio (values > 1 indicate increased delta likelihood):")
print(results[['dimension', 'odds_ratio']])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def create_social_dimensions_plot(results_df):
    plt.figure(figsize=(8, 10), dpi=300)
    
    # Sort dimensions by odds ratio
    results_df = results_df.sort_values('odds_ratio', ascending=True)
    
    # Create y-positions
    y_pos = np.arange(len(results_df))
    
    # Create scatter plot
    colors = ['red' if x <= 1 else 'blue' for x in results_df['odds_ratio']]
    plt.scatter(results_df['odds_ratio'], y_pos, color=colors)
    
    # Add vertical line at x=1
    plt.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    
    # Add labels
    plt.yticks(y_pos, results_df['dimension'])
    
    # Add odds ratio values
    for i, v in enumerate(results_df['odds_ratio']):
        plt.text(v, i, f' {v:.2f}', va='center')
    
    # Set axis limits and labels
    plt.xlim(0, 2.5)
    plt.xlabel('(a) Social dimensions')
    
    # Grid and formatting
    plt.grid(False)
    plt.tight_layout()
    plt.show()

# Create plot
create_social_dimensions_plot(results)

In [ ]:
def calculate_odds_ratios(data):
    dimensions = ['support', 'knowledge', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'identity', 'romance']
    
    results = []
    for dim in dimensions:
        # For comments (cleaned_body)
        body_median = data[f'{dim}_mean'].median()
        body_contingency = pd.crosstab(
            data['dimension'] == dim,
            data['has_delta']
        )
        
        # For posts (cleaned_text)
        text_median = data[f'{dim}_mean_text'].median()
        text_contingency = pd.crosstab(
            data['dimension_text'] == dim,
            data['has_delta']
        )
        
        # Calculate odds ratios
        try:
            body_or = (body_contingency.iloc[1,1] * body_contingency.iloc[0,0]) / \
                     (body_contingency.iloc[0,1] * body_contingency.iloc[1,0])
        except:
            body_or = np.nan
            
        try:
            text_or = (text_contingency.iloc[1,1] * text_contingency.iloc[0,0]) / \
                     (text_contingency.iloc[0,1] * text_contingency.iloc[1,0])
        except:
            text_or = np.nan
        
        results.append({
            'dimension': dim,
            'comment_odds_ratio': body_or,
            'post_odds_ratio': text_or
        })
    
    return pd.DataFrame(results)

# Calculate and display results
results = calculate_odds_ratios(data)
print("\nOdds Ratios (comparing delta vs non-delta):")
print(results.sort_values('comment_odds_ratio', ascending=False))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_odds_ratios(results_df):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), dpi=300)
    
    # Sort by odds ratios
    results_df_body = results_df.sort_values('comment_odds_ratio', ascending=True)
    results_df_text = results_df.sort_values('post_odds_ratio', ascending=True)
    
    # Comment plot
    y_pos = np.arange(len(results_df))
    colors1 = ['red' if x <= 1 else 'blue' for x in results_df_body['comment_odds_ratio']]
    ax1.scatter(results_df_body['comment_odds_ratio'], y_pos, color=colors1)
    ax1.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(results_df_body['semantic_dimension'])
    
    # Add values
    for i, v in enumerate(results_df_body['comment_odds_ratio']):
        if not np.isnan(v):
            ax1.text(v, i, f' {v:.2f}', va='center')
    
    ax1.set_xlim(0, max(results_df_body['comment_odds_ratio'].max(), 
                        results_df_text['post_odds_ratio'].max()) * 1.2)
    ax1.set_title('Comment (cleaned_body) Odds Ratios')
    ax1.grid(False)
    
    # Post plot
    y_pos = np.arange(len(results_df))
    colors2 = ['red' if x <= 1 else 'blue' for x in results_df_text['post_odds_ratio']]
    ax2.scatter(results_df_text['post_odds_ratio'], y_pos, color=colors2)
    ax2.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(results_df_text['dimension'])
    
    # Add values
    for i, v in enumerate(results_df_text['post_odds_ratio']):
        if not np.isnan(v):
            ax2.text(v, i, f' {v:.2f}', va='center')
    
    ax2.set_xlim(0, max(results_df_body['comment_odds_ratio'].max(), 
                        results_df_text['post_odds_ratio'].max()) * 1.2)
    ax2.set_title('Post (cleaned_text) Odds Ratios')
    ax2.grid(False)
    
    plt.tight_layout()
    plt.show()

# Create plots
plot_odds_ratios(results)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def calculate_odds_ratios(data):
    dimensions = ['support', 'knowledge', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'identity', 'romance']
    
    results = []
    for dim in dimensions:
        # For comments (cleaned_body dimension)
        dim_delta = len(data[(data['dimension'] == dim) & (data['has_delta'] == 1)])
        dim_no_delta = len(data[(data['dimension'] == dim) & (data['has_delta'] == 0)])
        other_delta = len(data[(data['dimension'] != dim) & (data['has_delta'] == 1)])
        other_no_delta = len(data[(data['dimension'] != dim) & (data['has_delta'] == 0)])
        
        # For posts (dimension_text)
        dim_text_delta = len(data[(data['dimension_text'] == dim) & (data['has_delta'] == 1)])
        dim_text_no_delta = len(data[(data['dimension_text'] == dim) & (data['has_delta'] == 0)])
        other_text_delta = len(data[(data['dimension_text'] != dim) & (data['has_delta'] == 1)])
        other_text_no_delta = len(data[(data['dimension_text'] != dim) & (data['has_delta'] == 0)])
        
        # Calculate odds ratios
        comment_or = ((dim_delta * other_no_delta) / 
                     (dim_no_delta * other_delta)) if (dim_no_delta * other_delta) != 0 else 0
        
        post_or = ((dim_text_delta * other_text_no_delta) / 
                  (dim_text_no_delta * other_text_delta)) if (dim_text_no_delta * other_text_delta) != 0 else 0
        
        results.append({
            'dimension': dim,
            'comment_odds_ratio': comment_or,
            'post_odds_ratio': post_or
        })
    
    return pd.DataFrame(results)

def plot_odds_ratios(results_df):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8), dpi=300)
    
    # Plot for comments
    comment_data = results_df.sort_values('comment_odds_ratio', ascending=True)
    y_pos = np.arange(len(comment_data))
    colors = ['red' if x <= 1 else 'blue' for x in comment_data['comment_odds_ratio']]
    
    ax1.scatter(comment_data['comment_odds_ratio'], y_pos, color=colors)
    ax1.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(comment_data['dimension'])
    
    for i, v in enumerate(comment_data['comment_odds_ratio']):
        if v > 0:
            ax1.text(v, i, f' {v:.2f}', va='center')
    
    ax1.set_title('Comment Odds Ratios')
    ax1.set_xlabel('Odds Ratio')
    
    # Plot for posts
    post_data = results_df.sort_values('post_odds_ratio', ascending=True)
    y_pos = np.arange(len(post_data))
    colors = ['red' if x <= 1 else 'blue' for x in post_data['post_odds_ratio']]
    
    ax2.scatter(post_data['post_odds_ratio'], y_pos, color=colors)
    ax2.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(post_data['dimension'])
    
    for i, v in enumerate(post_data['post_odds_ratio']):
        if v > 0:
            ax2.text(v, i, f' {v:.2f}', va='center')
    
    ax2.set_title('Post Odds Ratios')
    ax2.set_xlabel('Odds Ratio')
    
    plt.tight_layout()
    return fig

# Calculate and visualize
results = calculate_odds_ratios(data)
fig = plot_odds_ratios(results)
plt.show()

print("\nOdds Ratios Results:")
print(results.sort_values('comment_odds_ratio', ascending=False))

In [ ]:
def verify_dimension_counts(data):
    dimensions = ['knowledge', 'identity', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'support', 'romance']
    
    print("Comment Dimension Counts:")
    print("-" * 50)
    for dim in dimensions:
        delta = len(data[(data['dimension'] == dim) & (data['has_delta'] == 1)])
        no_delta = len(data[(data['dimension'] == dim) & (data['has_delta'] == 0)])
        total = len(data[data['dimension'] == dim])
        print(f"{dim}:")
        print(f"Delta: {delta}, No Delta: {no_delta}, Total: {total}")
        print(f"Delta Rate: {delta/total if total > 0 else 0:.4f}")
        print()

verify_dimension_counts(data)

In [ ]:
import numpy as np




# Add confidence intervals
results_df['error'] = np.sqrt(results_df['delta_rate'] * (1 - results_df['delta_rate']) / results_df['total_comments'])

# Plot with error bars
plt.figure(figsize=(8, 6))
sns.barplot(data=results_df, x='delta_rate', y='dimension', ci=None)
plt.errorbar(results_df['delta_rate'], results_df['dimension'], xerr=results_df['error'], fmt='none', c='black', capsize=5)
plt.title('Delta Rate by Primary Dimension with Error Bars')
plt.xlabel('Delta Rate')
plt.ylabel('Primary Dimension')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

def run_logistic_regression(data):
    dimensions = ['support', 'knowledge', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'identity', 'romance']
    
    X = data[[f'{dim}_mean' for dim in dimensions]].fillna(0)
    y = data['has_delta']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = LogisticRegression(random_state=42, class_weight='balanced')
    model.fit(X_train_scaled, y_train)
    
    coef_df = pd.DataFrame({
        'dimension': dimensions,
        'coefficient': model.coef_[0],
        'exp_coef': np.exp(model.coef_[0])
    })
    coef_df['abs_coefficient'] = abs(coef_df['coefficient'])
    coef_df = coef_df.sort_values('abs_coefficient', ascending=False)
    
    # Plotting
    plt.figure(figsize=(12, 6))
    colors = ['blue' if x > 0 else 'red' for x in coef_df['coefficient']]
    plt.bar(coef_df['dimension'], coef_df['coefficient'], color=colors)
    plt.xticks(rotation=45)
    plt.title('Dimension Coefficients in Predicting Delta')
    plt.tight_layout()
    plt.show()
    
    print("\nClassification Report:")
    y_pred = model.predict(X_test_scaled)
    print(classification_report(y_test, y_pred))
    
    print("\nFeature Importance:")
    print(coef_df[['dimension', 'coefficient', 'exp_coef']].round(4))
    
    return model, coef_df

model, coefficients = run_logistic_regression(data)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def enhanced_logistic_regression(data):
    dimensions = ['support', 'knowledge', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'identity', 'romance']
    
    X = data[[f'{dim}_mean' for dim in dimensions]].fillna(0)
    y = data['has_delta']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = LogisticRegression(random_state=42, class_weight='balanced')
    model.fit(X_train_scaled, y_train)
    
    # 1. ROC Curve
    plt.figure(figsize=(15, 5))
    plt.subplot(131)
    y_pred_proba = model.predict_proba(X_test_scaled)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.plot(fpr, tpr)
    plt.plot([0,1], [0,1], 'r--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve (AUC = {auc(fpr, tpr):.2f})')
    
    # 2. Feature Importance Plot
    plt.subplot(132)
    coef_df = pd.DataFrame({
        'dimension': dimensions,
        'coefficient': model.coef_[0],
        'abs_coef': abs(model.coef_[0])
    }).sort_values('abs_coef', ascending=True)
    
    colors = ['blue' if x > 0 else 'red' for x in coef_df['coefficient']]
    plt.barh(coef_df['dimension'], coef_df['coefficient'], color=colors)
    plt.title('Feature Coefficients')
    
    # 3. Confusion Matrix
    plt.subplot(133)
    cm = confusion_matrix(y_test, model.predict(X_test_scaled))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    plt.tight_layout()
    plt.show()
    
    # 4. Cross-validation scores
    cv_scores = cross_val_score(model, X, y, cv=5)
    print("\nCross-validation scores:", cv_scores)
    print("Mean CV score:", cv_scores.mean())
    
    # 5. Probability Distribution
    plt.figure(figsize=(8, 5))
    probas = model.predict_proba(X_test_scaled)[:,1]
    plt.hist(probas[y_test==0], bins=50, alpha=0.5, label='Non-Delta', density=True)
    plt.hist(probas[y_test==1], bins=50, alpha=0.5, label='Delta', density=True)
    plt.xlabel('Predicted Probability of Delta')
    plt.ylabel('Density')
    plt.title('Distribution of Predicted Probabilities')
    plt.legend()
    plt.show()

enhanced_logistic_regression(data)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_logistic_regression(data):
    dimensions = ['support', 'knowledge', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'identity', 'romance']
    
    X = data[[f'{dim}_mean' for dim in dimensions]].fillna(0)
    y = data['has_delta']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = LogisticRegression(random_state=42, class_weight='balanced')
    model.fit(X_train_scaled, y_train)
    
    # Get predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Calculate metrics
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    metrics = {
        'Accuracy': (tp + tn) / (tp + tn + fp + fn),
        'Precision': tp / (tp + fp) if (tp + fp) > 0 else 0,
        'Recall': tp / (tp + fn) if (tp + fn) > 0 else 0,
        'Specificity': tn / (tn + fp) if (tn + fp) > 0 else 0,
        'F1-Score': 2 * (tp / (tp + fp)) * (tp / (tp + fn)) / ((tp / (tp + fp)) + (tp / (tp + fn))) if tp > 0 else 0
    }
    
    # Plotting
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Confusion Matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1)
    ax1.set_title('Confusion Matrix')
    ax1.set_ylabel('True Label')
    ax1.set_xlabel('Predicted Label')
    
    # 2. Feature Importance
    coef_df = pd.DataFrame({
        'dimension': dimensions,
        'coefficient': model.coef_[0]
    }).sort_values('coefficient', ascending=True)
    
    colors = ['red' if x < 0 else 'blue' for x in coef_df['coefficient']]
    ax2.barh(coef_df['dimension'], coef_df['coefficient'], color=colors)
    ax2.set_title('Feature Coefficients')
    
    # 3. Probability Distribution
    ax3.hist(y_pred_proba[y_test==0], bins=50, alpha=0.5, label='Non-Delta', density=True)
    ax3.hist(y_pred_proba[y_test==1], bins=50, alpha=0.5, label='Delta', density=True)
    ax3.set_xlabel('Predicted Probability')
    ax3.set_ylabel('Density')
    ax3.set_title('Prediction Distribution')
    ax3.legend()
    
    # 4. ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    ax4.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc:.2f})')
    ax4.plot([0, 1], [0, 1], 'k--')
    ax4.set_xlabel('False Positive Rate')
    ax4.set_ylabel('True Positive Rate')
    ax4.set_title('ROC Curve')
    ax4.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\nMetrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    return model, metrics

# Run analysis
model, metrics = analyze_logistic_regression(data)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def enhanced_logistic_regression(data):
    dimensions = ['support', 'knowledge', 'conflict', 'power', 
                 'similarity', 'fun', 'status', 'trust', 
                 'identity', 'romance']
    
    X = data[[f'{dim}_mean' for dim in dimensions]].fillna(0)
    y = data['has_delta']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = LogisticRegression(random_state=42, class_weight='balanced')
    model.fit(X_train_scaled, y_train)
    
    # 1. ROC Curve
    plt.figure(figsize=(15, 5))
    plt.subplot(131)
    y_pred_proba = model.predict_proba(X_test_scaled)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.plot(fpr, tpr)
    plt.plot([0,1], [0,1], 'r--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve (AUC = {auc(fpr, tpr):.2f})')
    
    # 2. Feature Importance Plot
    plt.subplot(132)
    coef_df = pd.DataFrame({
        'dimension': dimensions,
        'coefficient': model.coef_[0],
        'abs_coef': abs(model.coef_[0])
    }).sort_values('abs_coef', ascending=True)
    
    colors = ['blue' if x > 0 else 'red' for x in coef_df['coefficient']]
    plt.barh(coef_df['dimension'], coef_df['coefficient'], color=colors)
    plt.title('Feature Coefficients')
    
    # 3. Confusion Matrix
    plt.subplot(133)
    cm = confusion_matrix(y_test, model.predict(X_test_scaled))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    plt.tight_layout()
    plt.show()
    
    # 4. Cross-validation scores
    cv_scores = cross_val_score(model, X, y, cv=5)
    print("\nCross-validation scores:", cv_scores)
    print("Mean CV score:", cv_scores.mean())
    
    # 5. Probability Distribution
    plt.figure(figsize=(8, 5))
    probas = model.predict_proba(X_test_scaled)[:,1]
    plt.hist(probas[y_test==0], bins=50, alpha=0.5, label='Non-Delta', density=True)
    plt.hist(probas[y_test==1], bins=50, alpha=0.5, label='Delta', density=True)
    plt.xlabel('Predicted Probability of Delta')
    plt.ylabel('Density')
    plt.title('Distribution of Predicted Probabilities')
    plt.legend()
    plt.show()

enhanced_logistic_regression(data)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_semantic_dimensions(data):
    # Calculate success rates by semantic dimension
    semantic_stats = pd.DataFrame({
        'total': data.groupby('semantic_dimension').size(),
        'deltas': data[data['has_delta'] == 1].groupby('semantic_dimension').size()
    }).fillna(0)
    
    semantic_stats['delta_rate'] = semantic_stats['deltas'] / semantic_stats['total']
    semantic_stats = semantic_stats.sort_values('delta_rate', ascending=False)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Delta rates
    semantic_stats['delta_rate'].plot(kind='bar', ax=ax1)
    ax1.set_title('Delta Success Rate by Semantic Dimension')
    ax1.set_ylabel('Delta Rate')
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    # Total counts
    semantic_stats['total'].plot(kind='bar', ax=ax2)
    ax2.set_title('Number of Comments by Semantic Dimension')
    ax2.set_ylabel('Count')
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return semantic_stats

stats = analyze_semantic_dimensions(data)
print("\nDimension Statistics:")
print(stats)

In [ ]:
def analyze_delta_success(data):
    # Get only delta comments
    delta_data = data[data['has_delta'] == 1]
    
    results = []
    dimensions = data['semantic_dimension'].unique()
    
    for dim in dimensions:
        # All comments for this dimension
        total = len(data[data['semantic_dimension'] == dim])
        # Delta comments for this dimension
        deltas = len(delta_data[delta_data['semantic_dimension'] == dim])
        
        # Calculate odds ratio
        a = deltas  # delta & this dimension
        b = total - deltas  # no delta & this dimension
        c = len(delta_data) - deltas  # delta & other dimensions
        d = len(data) - len(delta_data) - b  # no delta & other dimensions
        
        odds_ratio = (a * d) / (b * c) if (b * c) != 0 else np.nan
        
        results.append({
            'dimension': dim,
            'total_comments': total,
            'delta_comments': deltas,
            'success_rate': deltas/total if total > 0 else 0,
            'odds_ratio': odds_ratio
        })
    
    return pd.DataFrame(results).sort_values('odds_ratio', ascending=False)

results = analyze_delta_success(data)
print(results)

In [ ]:
def analyze_semantic_odds(data):
    dimensions = data['semantic_dimension'].unique()
    
    results = []
    for dim in dimensions:
        # Count deltas and non-deltas for this dimension
        dim_delta = len(data[(data['dimension'] == dim) & (data['has_delta'] == 1)])
        dim_no_delta = len(data[(data['dimension'] == dim) & (data['has_delta'] == 0)])
        
        # Count for other dimensions
        other_delta = len(data[(data['dimension'] != dim) & (data['has_delta'] == 1)])
        other_no_delta = len(data[(data['dimension'] != dim) & (data['has_delta'] == 0)])
        
        # Calculate odds ratio
        try:
            odds_ratio = (dim_delta * other_no_delta) / (dim_no_delta * other_delta)
        except:
            odds_ratio = np.nan
            
        results.append({
            'dimension': dim,
            'odds_ratio': odds_ratio,
            'total_comments': dim_delta + dim_no_delta,
            'delta_comments': dim_delta,
            'delta_rate': dim_delta/(dim_delta + dim_no_delta) if (dim_delta + dim_no_delta) > 0 else 0
        })
    
    results_df = pd.DataFrame(results).sort_values('odds_ratio', ascending=False)
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6), dpi=300)
    
    # Odds ratio plot
    colors = ['blue' if x > 1 else 'red' for x in results_df['odds_ratio']]
    bars = ax1.barh(results_df['dimension'], results_df['odds_ratio'], color=colors)
    ax1.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax1.set_xlabel('Odds Ratio')
    ax1.set_title('Impact of  Dimensions on Delta Likelihood')
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        if not np.isnan(width):
            ax1.text(width, i, f' {width:.2f}', va='center')
    
    # Comment counts plot
    bars = ax2.barh(results_df['dimension'], results_df['total_comments'])
    ax2.set_xlabel('Number of Comments')
    ax2.set_title('Total Comments per Semantic Dimension')
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax2.text(width, i, f' {int(width)}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    return results_df

results = analyze_semantic_odds(data)
print("\ngithub library embeddings Dimension Dimension Statistics:")
print(results[['dimension', 'odds_ratio', 'total_comments', 'delta_comments', 'delta_rate']])

In [ ]:
def analyze_semantic_odds(data):
    dimensions = data['semantic_dimension'].unique()
    
    results = []
    for dim in dimensions:
        # Count deltas and non-deltas for this dimension
        dim_delta = len(data[(data['semantic_dimension'] == dim) & (data['has_delta'] == 1)])
        dim_no_delta = len(data[(data['semantic_dimension'] == dim) & (data['has_delta'] == 0)])
        
        # Count for other dimensions
        other_delta = len(data[(data['semantic_dimension'] != dim) & (data['has_delta'] == 1)])
        other_no_delta = len(data[(data['semantic_dimension'] != dim) & (data['has_delta'] == 0)])
        
        # Calculate odds ratio
        try:
            odds_ratio = (dim_delta * other_no_delta) / (dim_no_delta * other_delta)
        except:
            odds_ratio = np.nan
            
        results.append({
            'dimension': dim,
            'odds_ratio': odds_ratio,
            'total_comments': dim_delta + dim_no_delta,
            'delta_comments': dim_delta,
            'delta_rate': dim_delta/(dim_delta + dim_no_delta) if (dim_delta + dim_no_delta) > 0 else 0
        })
    
    results_df = pd.DataFrame(results).sort_values('odds_ratio', ascending=False)
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6), dpi=300)
    
    # Odds ratio plot
    colors = ['blue' if x > 1 else 'red' for x in results_df['odds_ratio']]
    bars = ax1.barh(results_df['dimension'], results_df['odds_ratio'], color=colors)
    ax1.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax1.set_xlabel('Odds Ratio')
    ax1.set_title('Impact of Semantic Dimensions on Delta Likelihood')
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        if not np.isnan(width):
            ax1.text(width, i, f' {width:.2f}', va='center')
    
    # Comment counts plot
    bars = ax2.barh(results_df['dimension'], results_df['total_comments'])
    ax2.set_xlabel('Number of Comments')
    ax2.set_title('Total Comments per Semantic Dimension')
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax2.text(width, i, f' {int(width)}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    return results_df

results = analyze_semantic_odds(data)
print("\nSemantic Dimension Statistics:")
print(results[['dimension', 'odds_ratio', 'total_comments', 'delta_comments', 'delta_rate']])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_method_comparison():
    # Data
    zeroshot_data = {
        'trust': [1.553247, 331, 6, 0.018127],
        'similarity': [1.473048, 14716, 221, 0.015018],
        'power': [1.230356, 2732, 39, 0.014275],
        'identity': [1.074614, 158, 2, 0.012658],
        'support': [0.992591, 2647, 31, 0.011711],
        'conflict': [0.908932, 17105, 190, 0.011108],
        'knowledge': [0.858338, 2058, 21, 0.010204],
        'fun': [0.499125, 1167, 7, 0.005998],
        'status': [0.317925, 4443, 18, 0.004051],
        'romance': [0.000000, 10, 0, 0.000000]
    }
    
    embedding_data = {
        'knowledge': [2.888002, 29171, 448, 0.015358],
        'identity': [0.575262, 3955, 28, 0.007080],
        'power': [0.451791, 922, 5, 0.005423],
        'conflict': [0.426429, 8307, 47, 0.005658],
        'similarity': [0.388069, 1069, 5, 0.004677],
        'fun': [0.382972, 437, 2, 0.004577],
        'support': [0.000000, 508, 0, 0.000000],
        'status': [0.000000, 206, 0, 0.000000],
        'trust': [0.000000, 732, 0, 0.000000],
        'romance': [0.000000, 60, 0, 0.000000]
    }
    
    dimensions = list(zeroshot_data.keys())
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Odds Ratio Comparison
    zero_odds = [zeroshot_data[dim][0] for dim in dimensions]
    emb_odds = [embedding_data[dim][0] for dim in dimensions]
    
    ax1.barh(dimensions, zero_odds, alpha=0.6, label='Zero-shot', color='blue')
    ax1.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax1.set_title('Odds Ratios: Zero-shot')
    
    ax2.barh(dimensions, emb_odds, alpha=0.6, label='Embeddings', color='red')
    ax2.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax2.set_title('Odds Ratios: Embeddings')
    
    # Success Rate Comparison
    zero_rate = [zeroshot_data[dim][3] for dim in dimensions]
    emb_rate = [embedding_data[dim][3] for dim in dimensions]
    
    ax3.barh(dimensions, zero_rate, alpha=0.6)
    ax3.set_title('Success Rate: Zero-shot')
    
    ax4.barh(dimensions, emb_rate, alpha=0.6)
    ax4.set_title('Success Rate: Embeddings')
    
    plt.tight_layout()
    plt.show()
    
    # Print key differences
    print("\nKey differences between methods:")
    for dim in dimensions:
        if abs(zeroshot_data[dim][0] - embedding_data[dim][0]) > 0.5:
            print(f"\n{dim}:")
            print(f"Zero-shot OR: {zeroshot_data[dim][0]:.3f}")
            print(f"Embedding OR: {embedding_data[dim][0]:.3f}")
            print(f"Difference: {abs(zeroshot_data[dim][0] - embedding_data[dim][0]):.3f}")

plot_method_comparison()

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np

def evaluate_with_bert(data, batch_size=16):
    # Prepare data
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Encode labels
    le = LabelEncoder()
    labels = le.fit_transform(data['semantic_dimension'])
    
    # Tokenize texts
    encodings = tokenizer(data['cleaned_body'].tolist(), 
                         truncation=True, 
                         padding=True, 
                         max_length=512,
                         return_tensors='pt')
    
    # Create dataset
    dataset = TensorDataset(
        encodings['input_ids'],
        encodings['attention_mask'],
        torch.tensor(labels)
    )
    
    # Split data
    train_size = int(0.8 * len(dataset))
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, len(dataset) - train_size]
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Initialize model
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=len(le.classes_)
    )
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Training
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    
    for epoch in range(3):
        model.train()
        for batch in train_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            
            outputs = model(
                input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    
    # Evaluation
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    # Convert predictions back to labels
    pred_labels = le.inverse_transform(predictions)
    true_labels = le.inverse_transform(true_labels)
    
    # Compare with other methods
    comparison = pd.DataFrame({
        'Text': data['cleaned_body'].iloc[len(dataset)-len(predictions):].reset_index(drop=True),
        'BERT': pred_labels,
        'Zero_shot': data['semantic_dimension'].iloc[len(dataset)-len(predictions):].reset_index(drop=True),
        'Embedding': data['dimension'].iloc[len(dataset)-len(predictions):].reset_index(drop=True),
        'Has_Delta': data['has_delta'].iloc[len(dataset)-len(predictions):].reset_index(drop=True)
    })
    
    print("Classification Report (BERT):")
    print(classification_report(true_labels, pred_labels))
    
    return model, comparison

# Run evaluation
model, results = evaluate_with_bert(data)

# Analyze agreement between methods
agreement_stats = {
    'BERT_vs_Zero': (results['BERT'] == results['Zero_shot']).mean(),
    'BERT_vs_Embedding': (results['BERT'] == results['Embedding']).mean(),
    'Zero_vs_Embedding': (results['Zero_shot'] == results['Embedding']).mean()
}

print("\nAgreement between methods:")
for comparison, score in agreement_stats.items():
    print(f"{comparison}: {score:.3f}")

# Analyze success with deltas
for method in ['BERT', 'Zero_shot', 'Embedding']:
    success_rate = results[results['Has_Delta'] == 1][method].value_counts(normalize=True)
    print(f"\nDelta success distribution for {method}:")
    print(success_rate)

In [ ]:
def analyze_cop_impact(data):
    # First, check data types and content
    print("Checking data types:")
    print(data.dtypes)
    
    # Convert to datetime safely
    try:
        data['created_at'] = pd.to_datetime(data['created_at'])
        cop_date = pd.to_datetime('2023-12-13')
    except Exception as e:
        print(f"Error converting dates: {e}")
        return None
    
    # Check data ranges
    print("\nDate range in data:")
    print(f"Earliest: {data['created_at'].min()}")
    print(f"Latest: {data['created_at'].max()}")
    
    # Split data
    before_cop = data[data['created_at'] < cop_date]
    after_cop = data[data['created_at'] >= cop_date]
    
    print(f"\nComments before COP: {len(before_cop)}")
    print(f"Comments after COP: {len(after_cop)}")
    
    def get_dimension_stats(df, period):
        if len(df) == 0:
            print(f"Warning: No data for period {period}")
            return pd.DataFrame()
            
        dim_stats = []
        for dim in df['dimension'].unique():
            dim_data = df[df['dimension'] == dim]
            stats = {
                'period': period,
                'dimension': dim,
                'total_comments': len(dim_data),
                'delta_comments': len(dim_data[dim_data['has_delta'] == 1]),
                'delta_rate': len(dim_data[dim_data['has_delta'] == 1]) / len(dim_data) if len(dim_data) > 0 else 0
            }
            dim_stats.append(stats)
        return pd.DataFrame(dim_stats)
    
    before_stats = get_dimension_stats(before_cop, 'Before COP28')
    after_stats = get_dimension_stats(after_cop, 'After COP28')
    
    if len(before_stats) == 0 or len(after_stats) == 0:
        print("Error: Insufficient data for comparison")
        return None
    
    # Ensure same dimensions in both periods
    all_dims = sorted(list(set(before_stats['dimension'].unique()) | 
                         set(after_stats['dimension'].unique())))
    
    # Plot with error handling
    try:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        x = np.arange(len(all_dims))
        width = 0.35
        
        # Ensure data exists for all dimensions
        before_rates = [before_stats[before_stats['dimension'] == dim]['delta_rate'].iloc[0] 
                       if dim in before_stats['dimension'].values else 0 
                       for dim in all_dims]
        after_rates = [after_stats[after_stats['dimension'] == dim]['delta_rate'].iloc[0] 
                      if dim in after_stats['dimension'].values else 0 
                      for dim in all_dims]
        
        ax1.bar(x - width/2, before_rates, width, label='Before COP28')
        ax1.bar(x + width/2, after_rates, width, label='After COP28')
        ax1.set_xticks(x)
        ax1.set_xticklabels(all_dims, rotation=45)
        ax1.set_title('Delta Success Rate by Period')
        ax1.legend()
        
        # Same for comment counts
        before_counts = [before_stats[before_stats['dimension'] == dim]['total_comments'].iloc[0] 
                        if dim in before_stats['dimension'].values else 0 
                        for dim in all_dims]
        after_counts = [after_stats[after_stats['dimension'] == dim]['total_comments'].iloc[0] 
                       if dim in after_stats['dimension'].values else 0 
                       for dim in all_dims]
        
        ax2.bar(x - width/2, before_counts, width, label='Before COP28')
        ax2.bar(x + width/2, after_counts, width, label='After COP28')
        ax2.set_xticks(x)
        ax2.set_xticklabels(all_dims, rotation=45)
        ax2.set_title('Comment Distribution by Period')
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error in plotting: {e}")
    
    # Calculate changes
    try:
        changes = pd.merge(before_stats, after_stats, on='dimension', suffixes=('_before', '_after'))
        changes['delta_rate_change'] = changes['delta_rate_after'] - changes['delta_rate_before']
        changes['distribution_change'] = (changes['total_comments_after'] / after_stats['total_comments'].sum()) - \
                                       (changes['total_comments_before'] / before_stats['total_comments'].sum())
        
        return changes.sort_values('delta_rate_change', ascending=False)
    except Exception as e:
        print(f"Error calculating changes: {e}")
        return None

# Run analysis with error handling
try:
    results = analyze_cop_impact(data)
    if results is not None:
        print("\nChanges in dimension effectiveness after COP28:")
        print(results[['dimension', 'delta_rate_change', 'distribution_change']])
except Exception as e:
    print(f"Analysis failed: {e}")

In [ ]:
def analyze_cop_impact(data):
    # Convert to datetime and standardize timezone
    data['created_at'] = pd.to_datetime(data['created_at']).dt.tz_localize(None)
    cop_date = pd.to_datetime('2023-12-13')
    
    print("Date range in data:")
    print(f"Earliest: {data['created_at'].min()}")
    print(f"Latest: {data['created_at'].max()}")
    
    # Split data
    before_cop = data[data['created_at'] < cop_date]
    after_cop = data[data['created_at'] >= cop_date]
    
    print(f"\nComments before COP: {len(before_cop)}")
    print(f"Comments after COP: {len(after_cop)}")
    
    def get_dimension_stats(df, period):
        if len(df) == 0:
            print(f"Warning: No data for period {period}")
            return pd.DataFrame()
            
        dim_stats = []
        for dim in sorted(df['dimension'].unique()):
            dim_data = df[df['dimension'] == dim]
            stats = {
                'period': period,
                'dimension': dim,
                'total_comments': len(dim_data),
                'delta_comments': len(dim_data[dim_data['has_delta'] == 1]),
                'delta_rate': len(dim_data[dim_data['has_delta'] == 1]) / len(dim_data) if len(dim_data) > 0 else 0
            }
            dim_stats.append(stats)
        return pd.DataFrame(dim_stats)
    
    before_stats = get_dimension_stats(before_cop, 'Before COP28')
    after_stats = get_dimension_stats(after_cop, 'After COP28')
    
    # Plot comparisons
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    dimensions = sorted(list(set(before_stats['dimension'].unique()) | 
                           set(after_stats['dimension'].unique())))
    x = np.arange(len(dimensions))
    width = 0.35
    
    # Delta rates
    before_rates = [before_stats[before_stats['dimension'] == dim]['delta_rate'].iloc[0] 
                   if dim in before_stats['dimension'].values else 0 
                   for dim in dimensions]
    after_rates = [after_stats[after_stats['dimension'] == dim]['delta_rate'].iloc[0] 
                  if dim in after_stats['dimension'].values else 0 
                  for dim in dimensions]
    
    bars1 = ax1.bar(x - width/2, before_rates, width, label='Before COP28')
    bars2 = ax1.bar(x + width/2, after_rates, width, label='After COP28')
    ax1.set_xticks(x)
    ax1.set_xticklabels(dimensions, rotation=45, ha='right')
    ax1.set_title('Delta Success Rate by Period')
    ax1.legend()
    
    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom')
    
    # Comment distribution
    before_counts = [before_stats[before_stats['dimension'] == dim]['total_comments'].iloc[0] 
                    if dim in before_stats['dimension'].values else 0 
                    for dim in dimensions]
    after_counts = [after_stats[after_stats['dimension'] == dim]['total_comments'].iloc[0] 
                   if dim in after_stats['dimension'].values else 0 
                   for dim in dimensions]
    
    bars3 = ax2.bar(x - width/2, before_counts, width, label='Before COP28')
    bars4 = ax2.bar(x + width/2, after_counts, width, label='After COP28')
    ax2.set_xticks(x)
    ax2.set_xticklabels(dimensions, rotation=45, ha='right')
    ax2.set_title('Comment Distribution by Period')
    ax2.legend()
    
    # Add value labels
    for bars in [bars3, bars4]:
        for bar in bars:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}',
                    ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate changes
    changes = pd.merge(before_stats, after_stats, on='dimension', suffixes=('_before', '_after'))
    changes['delta_rate_change'] = changes['delta_rate_after'] - changes['delta_rate_before']
    changes['distribution_change'] = (changes['total_comments_after'] / after_stats['total_comments'].sum()) - \
                                   (changes['total_comments_before'] / before_stats['total_comments'].sum())
    
    return changes.sort_values('delta_rate_change', ascending=False)

# Run analysis
results = analyze_cop_impact(data)
print("\nChanges in dimension effectiveness after COP28:")
print(results[['dimension', 'delta_rate_change', 'distribution_change']])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.cluster import KMeans

def analyze_topics(data, n_topics=5):
    # LDA Analysis
    vectorizer = CountVectorizer(max_features=1000, stop_words='english')
    doc_term_matrix = vectorizer.fit_transform(data['cleaned_body'])
    
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda_output = lda.fit_transform(doc_term_matrix)
    
    # Print top words per topic
    feature_names = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-10:-1]]
        print(f"\nTopic {topic_idx + 1} top words:")
        print(", ".join(top_words))
    
    # BERT embeddings
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
        outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].detach().numpy()
    
    # Get embeddings for a sample of comments
    sample_size = min(1000, len(data))
    sample_data = data.sample(n=sample_size, random_state=42)
    embeddings = np.vstack([get_bert_embedding(text) for text in sample_data['cleaned_body']])
    
    # Cluster BERT embeddings
    kmeans = KMeans(n_clusters=n_topics, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    
    # Analyze clusters
    cluster_stats = []
    for cluster in range(n_topics):
        cluster_data = sample_data[clusters == cluster]
        stats = {
            'cluster': cluster,
            'size': len(cluster_data),
            'delta_rate': cluster_data['has_delta'].mean(),
            'main_dimension': cluster_data['dimension'].mode().iloc[0]
        }
        cluster_stats.append(stats)
    
    return pd.DataFrame(cluster_stats)

# Run analysis
topic_results = analyze_topics(data)
print("\nCluster Statistics:")
print(topic_results)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

def advanced_topic_analysis(data, n_topics=5):
    # 1. Improved LDA Analysis
    vectorizer = CountVectorizer(
        max_features=2000, 
        stop_words='english',
        ngram_range=(1, 2)  # Include bigrams
    )
    doc_term_matrix = vectorizer.fit_transform(data['cleaned_body'])
    
    lda = LatentDirichletAllocation(
        n_components=n_topics,
        random_state=42,
        learning_method='batch'
    )
    
    # Get document-topic distributions
    doc_topics = lda.fit_transform(doc_term_matrix)
    
    # Store results
    feature_names = vectorizer.get_feature_names_out()
    topics_data = []
    
    for topic_idx, topic in enumerate(lda.components_):
        # Get top words with their weights
        top_words_idx = topic.argsort()[:-20:-1]
        topic_words = [(feature_names[i], topic[i]) for i in top_words_idx]
        
        # Get documents strongly associated with this topic
        topic_docs = data[doc_topics[:, topic_idx] > np.percentile(doc_topics[:, topic_idx], 75)]
        
        topics_data.append({
            'topic_id': topic_idx,
            'top_words': topic_words,
            'n_documents': len(topic_docs),
            'delta_rate': topic_docs['has_delta'].mean(),
            'dominant_dimension': topic_docs['dimension'].mode().iloc[0],
            'dimension_distribution': topic_docs['dimension'].value_counts(normalize=True)
        })
    
    # Visualizations
    fig = plt.figure(figsize=(20, 15))
    
    # 1. Topic Word Importance
    ax1 = plt.subplot(221)
    for topic in topics_data:
        word_weights = [w[1] for w in topic['top_words'][:10]]
        ax1.plot(range(10), word_weights, marker='o', label=f'Topic {topic["topic_id"]}')
    ax1.set_xlabel('Top Words (by rank)')
    ax1.set_ylabel('Word Importance')
    ax1.set_title('Word Weight Distribution per Topic')
    ax1.legend()
    
    # 2. Delta Success by Topic
    ax2 = plt.subplot(222)
    delta_rates = [t['delta_rate'] for t in topics_data]
    sns.barplot(x=range(n_topics), y=delta_rates, ax=ax2)
    ax2.set_xlabel('Topic')
    ax2.set_ylabel('Delta Rate')
    ax2.set_title('Delta Success Rate by Topic')
    
    # 3. Topic-Dimension Heatmap
    ax3 = plt.subplot(212)
    dimension_matrix = pd.DataFrame([t['dimension_distribution'] for t in topics_data])
    sns.heatmap(dimension_matrix, annot=True, fmt='.2f', cmap='YlOrRd', ax=ax3)
    ax3.set_title('Dimension Distribution across Topics')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed topic information
    print("\nDetailed Topic Analysis:")
    for topic in topics_data:
        print(f"\nTopic {topic['topic_id'] + 1}")
        print("-" * 50)
        print("Top 10 Words:")
        for word, weight in topic['top_words'][:10]:
            print(f"{word}: {weight:.4f}")
        print(f"\nNumber of Documents: {topic['n_documents']}")
        print(f"Delta Rate: {topic['delta_rate']:.4f}")
        print(f"Dominant Dimension: {topic['dominant_dimension']}")
        print("\nDimension Distribution:")
        for dim, prop in topic['dimension_distribution'].items():
            print(f"{dim}: {prop:.3f}")
    
    return topics_data

# Run analysis
topics_data = advanced_topic_analysis(data)